In [8]:
%reload_ext autoreload
%autoreload 2

In [183]:
import os
import pprint
import io

import pymongo
import json
import pandas as pd

import climatools.lblnew.setup_bestfit as setup_bestfit
import climatools.lblnew.runrecord_bestfit as runrecord_bestfit
import climatools.lblnew.pipeline as pipeline


In [206]:
client = pymongo.MongoClient('localhost', 27017, connect=False)

In [207]:
db = client.lblnew

In [208]:
collection = db.bestfit_lw

In [233]:
params = runrecord.params()

In [234]:
def make_doc(param=None):
    '''
    Makes a Mongodb document for an lblnew-bestfit run.
    
    Parameters
    ----------
    param: dict
        lblnew-bestfit input parameters.
    doc: dict
        lblnew-bestfit input (param) and output (flux and cooling rate)
    '''
#    fnames = ['output_wfluxg.dat', 'output_wcoolrg.dat',
#              'output_fluxg.dat', 'output_coolrg.dat',
#              'output_flux.dat', 'output_coolr.dat']

    dir_fortran = pipeline.get_dir_case(param=param, 
                                        setup=setup_bestfit)
    
    fnames = [n for n in os.listdir(dir_fortran) if n.endswith('.dat')]
    
    s_outputs = []
    for fname in fnames:
        with open(os.path.join(dir_fortran, fname), 
                  mode='r', encoding='utf-8') as f:
            s = f.read()
        
        s_outputs.append(s)
    
    doc = {}
    doc['param'] = param
    
    for fname, s in zip(fnames, s_outputs):
        doc[fname.split('.')[0]] = s
    
    return doc



def make_query(param=None):
    '''
    Returns the MongoDB query for a lblnew-bestfit 
    run's document.
    
    Parameters
    ----------
    param: dict
    lblnew-bestfit input parameters.
    '''
    return {'param.' + name: value for name, value in param.items()}
    

In [243]:
docs = (make_doc(param=param) for param in params)
collection.insert_many(docs)

In [245]:
#collection.delete_many({'param.atmpro': 'trp'})

In [244]:
collection.count()

96